# Oracle MCP Agent Workshop

## Building Agentic Workflows with the Model Context Protocol (MCP)

### Introduction

This notebook demonstrates how to build a modular, enterprise-grade email assistant using the **Oracle Model Context Protocol (MCP)** as the backbone for all agent-tool orchestration. MCP empowers you to decouple business logic into reusable "tools" that are securely registered and dynamically accessed by GenAI agents.

**With MCP, your agents can:**
- Search and summarize organization-specific documents through Retrieval-Augmented Generation (RAG)
- Look up email recipients from authoritative company directories
- Draft, send, and manage emails—all via orchestrated tool calls rather than hardcoded logic

**In this workshop, you'll learn how to:**
1. Set up a robust, MCP-based environment for scalable agent workflows using Oracle, LangChain, and MCP
2. Register custom business and document tools using the MCP Tool Server (`@mcp.tool`)
3. Connect notebooks and applications to the MCP Tool Server, enabling dynamic discovery and invocation of registered tools
4. Perform advanced document ingestion, semantic search, and retrieval using RAG—all as MCP tools
5. Orchestrate complex, multi-step tasks via MCP by combining GenAI agent planning with enterprise automation tools

Throughout this notebook, every agent action is backed by the Model Context Protocol (MCP), ensuring modularity, transparency, and ease of extension as your enterprise needs evolve.

Let's get started with MCP!

# Section 1: Environment Setup/ Improrts and Database Connnection

We first set up our Python environment and import the necessary libraries, including LangChain components, OracleDB drivers, and the MCP (Modular Command Processor) platform.

- We use OracleDB Python drivers, **not** the deprecated `cx_Oracle`, to support vector search.
- All tools and configurations are version-controlled and reproducible in this notebook.

## Imports and Configuration

In [1]:
import os
import nest_asyncio
import subprocess
from dotenv import load_dotenv
from tools9 import DatabaseOperations


# Enable nested asyncio loops for Jupyter
nest_asyncio.apply()

# Load .env with Oracle etc.
load_dotenv()

# (Optional) Silence HuggingFace tokenizer warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Oracle Database Connection

Connecting to Oracle database using the database username and password which are stored as environment variables (in .env file on linux)

In [2]:
db_ops = DatabaseOperations()
connected = db_ops.connect()
print("✅ Connected!" if connected else "❌ Connection Failed")


✅ Connected!


## 2. Start the MCP Tool Server in the Background

Before your GenAI agent can access any tools, you must start the **MCP Tool Server**, which acts as a registry and secure communication bridge between your modular tools and any client (notebooks, apps, or agents). With MCP, you can update, restart, or extend your tool ecosystem independently of your agent logic. This ensures that as team or enterprise needs change, your agents always have access to the latest, most reliable automation tools.

The MCP Tool Server is responsible for:
- Registering all your custom tools (with `@mcp.tool()` decorators)
- Handling communication between your notebook’s agent and the available tools

**How it works:**
- Start the tool server as a separate background process.
- The server listens for requests—such as RAG searches, recipient lookups, or email drafts—from your agent running in the notebook.

This approach ensures modularity: you can update your tools or restart the server independently of your notebook session.

**In this notebook, we will:**
- Use Python’s `subprocess` module to launch `server.py` in the background.
- Display the server code here for transparency and reproducibility.

**Tip:** If you modify your tools in `server.py`, restart the server process to load the changes.

In [4]:
import time
import subprocess
import sys

python_path = sys.executable  # dynamically picks current kernel's Python path
server_process = subprocess.Popen([python_path, 'server.py'])

print(f"MCP server.py started in background with PID {server_process.pid}")
time.sleep(2)

NCP server.py started in background with PID 57337


## 3. Review Your MCP Tool Server (`server.py`)

With MCP, all tools are explicitly registered and auditable. Here, we'll inspect the code that defines and registers each modular tool—such as `rag_search`, `lookup_recipients`, and email orchestration—ensuring every step in the agent workflow is driven by a transparent, MCP-managed contract between code and agent.

This practice is central to MCP: tools are not just functions, but versioned, documented, testable building blocks discoverable at runtime by any MCP-compatible agent.

In [5]:
from IPython.display import Markdown, display

with open("server.py", "r") as f:
    server_code = f.read()
display(Markdown(f"```python\n{server_code}\n```"))

```python
from mcp.server.fastmcp import FastMCP
from tools9 import DatabaseOperations, fetch_recipients, send_email_function, extract_email_data_from_response

from tools9 import fetch_recipients, send_email_function, chunks_to_docs_wrapper
from typing import List
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores.oraclevs import OracleVS
import os
mcp = FastMCP("EmailAssistant")

global_vector_store = None

def set_vector_store(store: OracleVS):
    global global_vector_store
    global_vector_store = store


def get_vector_store():
    global global_vector_store   # ← Add this line
    if global_vector_store:
        return global_vector_store

    
    try:
        db_ops = DatabaseOperations()
        if not db_ops.connect():
            raise Exception("DB connect failed")

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

        global_vector_store = OracleVS(
        embedding_function=embeddings,
        client=db_ops.connection,
        table_name="MY_DEMO4",
        distance_strategy=DistanceStrategy.COSINE,
        )
        return global_vector_store
    except Exception as e:
        print("VectorStore init failed:", e)
        return None



@mcp.tool()
def lookup_recipients(name: str):
    return fetch_recipients(name)

# @mcp.tool()
# def prepare_and_send_email(to: str, subject: str, message: str):
#     return send_email_function({"to": to, "subject": subject, "message": message})


@mcp.tool()
def oracle_connect() -> str:
    """
    Checks and returns Oracle DB connection status.
    """
    try:
        db_ops = DatabaseOperations()
        if db_ops.connect():
            print("Oracle connection successful!")
            return db_ops.connection
        return None
    except Exception as e:
        print(f"Oracle connection failed: {str(e)}")
        return None    

@mcp.tool()
def extract_email_fields_from_response(response_text: str) -> dict:
    """
    Extracts email fields (to, subject, message) from an AI-generated response.

    Input:
    - response_text: A string containing the AI assistant's output.

    Output:
    - A dictionary with keys: "to", "subject", "message"
    """
    try:
        return extract_email_data_from_response(response_text)
    except Exception as e:
        return {"error": f"Failed to extract email data: {str(e)}"}


@mcp.tool()
def store_text_chunks(file_path: str) -> str:
    """Split text and store as embeddings in Oracle Vector Store"""
    try:
        db_ops = DatabaseOperations()
        
        if not db_ops.connect():
            return "❌ Oracle connection failed."

        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            raw_text = f.read()

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            chunks = text_splitter.split_text(raw_text)
            file_name = os.path.basename(file_path)
            docs = [
                chunks_to_docs_wrapper({'id': f"{file_name}_{i}", 'link': f"{file_name} - Chunk {i}", 'text': chunk})
                for i, chunk in enumerate(chunks)
            ]


            embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
            vector_store = OracleVS.from_documents(
                docs, embeddings, client=db_ops.connection,
                table_name="MY_DEMO4", distance_strategy=DistanceStrategy.COSINE)
            
            # OracleVS(
            #     embedding_function=embeddings,
            #     client=db_ops.connection,
            #     table_name="MY_DEMO4",
            #     distance_strategy=DistanceStrategy.COSINE,
            # )

            set_vector_store(vector_store)

            return f"✅ Stored {len(docs)} chunks from {file_name}"

    except Exception as e:
        return f"❌ Error: {str(e)}"

@mcp.tool()
def rag_search(query: str) -> str:
    """
    Retrieve relevant information from user-uploaded documents stored in the Oracle Vector Store.

    Use this tool whenever a user asks a question that may be answered from the uploaded documents
    (e.g., HR policy files, contracts, technical manuals, PDF uploads, etc.).

    The tool performs a semantic similarity search over the embedded document chunks and returns
    the top 5 most relevant text snippets.

    Input:
    - A natural language question or topic from the user.

    Output:
    - A formatted string combining the most relevant document excerpts.

    Examples:
    - "What is the leave policy for new employees?"
    - "Summarize the refund terms in the uploaded contract"
    - "Find safety precautions mentioned in the manual"
    """
    try:
        # Load vector store (or access from persistent source if needed)
        vector_store = get_vector_store()
        if vector_store is None:
            return "❌ No documents have been indexed yet."

        docs = vector_store.similarity_search(query, k=5)
        return "\n".join([doc.page_content for doc in docs])
    except Exception as e:
        return f"❌ Error during document search: {str(e)}"


if __name__ == "__main__":
    print(" Starting MCP Agentic Server ...")
    mcp.run(transport="stdio")


```

## 4. MCP Agent & GenAI Setup

Once your MCP Tool Server is running, agents and notebooks connect to it to dynamically discover and access your suite of tools.

Here, we pair a large language model (LLM) with the Model Context Protocol to enable agents that not only chat, but autonomously plan and execute multi-step business workflows—by invoking your modular MCP tools based on language, context, and user goals.

In [6]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain.schema import HumanMessage

# Set up the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# This param is required by the package, but since our server is already running, we'll set connect=False below
server_params = StdioServerParameters(command="python", args=["server.py"])

async def run_mcp_agent(prompt):
    # Connect to the already-launched MCP tool server (stdin/stdout)
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await load_mcp_tools(session)
            agent = create_react_agent(llm, tools)
            response = await agent.ainvoke({
                "messages": [HumanMessage(prompt)]
            })
            return response["messages"][-1].content

## 5. Index a Test Document via MCP Tool

MCP treats every operation—document ingestion, RAG search, email sending—as a tool. Even document indexing is performed through a registered MCP tool, ensuring a unified interface for all automation steps. This design makes it easy to add, update, or re-use business logic across workflows.

In [7]:
# Create a simple policy file
test_file_path = "sample_policy.txt"

# Ask the MCP agent to index the file using the document chunking tool
response = await run_mcp_agent("""
Please index the file sample_policy.txt placed in the current directory.
""")

print(response)


The file `sample_policy.txt` has been successfully indexed and stored as 2 chunks. If you have any questions or need further assistance, feel free to ask!


## 6. Run an End-to-End Agentic Workflow with MCP

Providing a single natural-language prompt, the GenAI agent—using MCP—remotely discovers, plans, and executes a multi-tool workflow. MCP ensures each invocation, whether for document search or business communication, is managed and auditable, making your agent trustworthy, extensible, and enterprise-ready.

In [8]:
user_query = (
    "Find the leave policy from the indexed documents and email HR with the summary."
)
result = await run_mcp_agent(user_query)
print("AGENT RESPONSE:\n", result)

AGENT RESPONSE:
 It seems I couldn't find a specific email address for the HR department in the system. However, based on the document, you can send the email to `hr@company.com`. Here's the email content you can use:

---

**Subject:** Summary of Employee Leave Policy

**To:** hr@company.com

Dear HR Team,

I hope this message finds you well. I am writing to provide a summary of the current Employee Leave Policy as per the latest document review:

1. **Annual Leave**: Full-time employees are entitled to 15 working days of paid annual leave per year, accruing at 1.25 days per month. A maximum of 5 unused days can be carried over to the next year.

2. **Sick Leave**: Employees receive 10 days of paid sick leave annually, with a medical certificate required for absences over 3 consecutive days. Unused sick leave does not carry over.

3. **Parental Leave**: Maternity leave is 12 weeks paid, paternity leave is 4 weeks paid, and adoption leave is 8 weeks paid.

4. **Special Circumstances**:

## 7. MCP Tool Demonstrations: RAG, Recipient Lookup, & Email Drafts

In this section, you'll directly experience the power of MCP as a modular orchestration layer for enterprise workflows.

- **RAG Search via MCP:** The agent leverages a registered MCP tool to perform Retrieval-Augmented Generation, sourcing accurate answers from your indexed organizational data—no direct coding required!
- **Recipient Lookup via MCP:** Using MCP’s tool abstraction, agents can securely and reliably query up-to-date company directory information from a single source of truth.
- **Email Automation via MCP Toolchain:** Thanks to MCP, email drafting and sending is abstracted into a robust, reusable tool—invocable by any compliant agent with proper permissions.

In your prompts, you're not just querying a language model—you're activating powerful, encapsulated automation via the Model Context Protocol tool network.


In [9]:
print("---\nRAG Search Example:")
result = await run_mcp_agent("What is the leave policy?")
print(result)

print("---\nRecipient Lookup Example:")
result = await run_mcp_agent("Find the email for Ashu.")
print(result)

print("---\nEmail Draft Example:")
result = await run_mcp_agent("Send a draft email about annual leave policy to Ashu.")
print(result)

---
RAG Search Example:
The leave policy includes the following key sections:

### Annual Leave Entitlements
- **Full-time employees** are entitled to 15 working days of paid annual leave per calendar year.
- Leave accrues at a rate of 1.25 days per month of service.
- A maximum of 5 unused leave days can be carried over to the next calendar year.

### Sick Leave
- Employees receive 10 days of paid sick leave annually.
- A medical certificate is required for absences exceeding 3 consecutive days.
- Unused sick leave does not carry over to the next year.

### Parental Leave
- **Maternity Leave**: 12 weeks paid leave for new mothers.
- **Paternity Leave**: 4 weeks paid leave for new fathers.
- **Adoption Leave**: 8 weeks paid leave for adoptive parents.

### Special Circumstances
- **Bereavement Leave**: 5 days for immediate family members.
- **Jury Duty**: Full pay for the duration of service.
- **Military Leave**: Protected unpaid leave for reservists.

### Contact Information
- **HR D

## 8. Demonstrate Domain Expertise Extraction via RAG & MCP

The Model Context Protocol doesn't just automate steps—it lets you encode, evolve, and reuse domain knowledge as tools. With MCP, your agent can surface key product, technical, or policy features directly from live document data using RAG and summarization—all through the modular MCP interface.

This capability is critical for:
- Delivering expert answers grounded in real company content
- Breaking silos so every user can access up-to-date enterprise knowledge via MCP-driven agents
- Setting the foundation for AI-powered onboarding, training, and decision support—all orchestrated through MCP


In [10]:
prompt = "List 5 new features of Oracle 23ai from the indexed documents."
result = await run_mcp_agent(prompt)
print(result)

The search did not return relevant information about the new features of Oracle 23ai. If you have specific documents or details about Oracle 23ai, please provide them, or I can attempt another search with different parameters.


## 9. Shutting Down the MCP Tool Server

When you're finished running experiments or workflows, it’s important to terminate the MCP Tool Server initialized earlier.


In [13]:
server_process.terminate()
print("Stopped MCP server process.")

Stopped MCP server process.
